In [ ]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import textwrap
import re
import os
from IPython.display import display, Markdown

output_files = [
        "exp-09-25_114848-LLaMEA-gpt-5-nano-ELA-Separable_GlobalLocal-sharing",
        "exp-09-25_115914-LLaMEA-gpt-5-nano-ELA-Separable_Multimodality-sharing",
        "exp-09-25_121517-LLaMEA-gpt-5-nano-ELA-Separable_Homogeneous-sharing",
        "exp-09-25_122101-LLaMEA-gpt-5-nano-ELA-Separable-sharing",
        "exp-09-25_122848-LLaMEA-gpt-5-nano-ELA-GlobalLocal_Multimodality-sharing",
        "exp-09-25_132136-LLaMEA-gpt-5-nano-ELA-GlobalLocal_Homogeneous-sharing",
        "exp-09-25_144046-LLaMEA-gpt-5-nano-ELA-GlobalLocal-sharing",
        "exp-09-25_145409-LLaMEA-gpt-5-nano-ELA-Multimodality_Homogeneous-sharing",
        "exp-09-25_165258-LLaMEA-gpt-5-nano-ELA-Multimodality-sharing",
        "exp-09-25_165843-LLaMEA-gpt-5-nano-ELA-Homogeneous-sharing",
        "exp-09-26_120435-LLaMEA-gpt-5-nano-ELA-NOT Homogeneous_Separable-sharing",
        "exp-09-26_121734-LLaMEA-gpt-5-nano-ELA-NOT Homogeneous_GlobalLocal-sharing",
        "exp-09-26_122528-LLaMEA-gpt-5-nano-ELA-NOT Homogeneous_Multimodality-sharing",
        "exp-09-26_123417-LLaMEA-gpt-5-nano-ELA-NOT Homogeneous-sharing",
        "exp-10-30_094616-LLaMEA-gpt-5-nano-ELA-NOT Basins_Separable-sharing",
        "exp-10-30_103416-LLaMEA-gpt-5-nano-ELA-NOT Basins_GlobalLocal-sharing",
        "exp-10-30_115015-LLaMEA-gpt-5-nano-ELA-NOT Basins_Multimodality-sharing", 
        "exp-10-30_120126-LLaMEA-gpt-5-nano-ELA-NOT Basins-sharing" ,
        "exp-10-30_121257-LLaMEA-gpt-5-nano-ELA-Basins_Separable-sharing" ,
        "exp-10-30_123650-LLaMEA-gpt-5-nano-ELA-Basins_GlobalLocal-sharing", 
        "exp-10-30_150153-LLaMEA-gpt-5-nano-ELA-Basins_Multimodality-sharing", 
        "exp-10-30_182432-LLaMEA-gpt-5-nano-ELA-Basins-sharing"
    ]



# --- SETTINGS ---
bounds = (-5, 5)
dim = 2  # visualize in 2D
grid_points = 200  # resolution


for f in output_files:
    file_path = f"/home/neocortex/repos/LLaMEA-ELA/LLaMEA/basinsattribution-main/outputs/{f}.jsonl" 

    filename = os.path.basename(file_path)
    match = re.search(r"-ELA-(.*?)(?:-|\-sharing\.jsonl$)", filename)
    title = match.group(1).replace("_", " ") if match else filename

    # --- DISPLAY MARKDOWN TITLE ---
    display(Markdown(f"## {title}"))

    # --- LOAD JSON LINES ---
    with open(file_path, "r") as f:
        data = [json.loads(line) for line in f if line.strip()]

    n = len(data)
    cols = 5
    rows = math.ceil(n / cols)

    # --- SETUP FIGURE ---
    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
    axes = axes.flatten()

    # --- EVALUATE AND PLOT EACH LANDSCAPE ---
    # This is also an example of how you can load the Python code for each of the generated landscapes easily.
    x = np.linspace(*bounds, grid_points)
    y = np.linspace(*bounds, grid_points)
    X, Y = np.meshgrid(x, y)

    for i, entry in enumerate(data):
        ax = axes[i]
        code = entry["code"]
        try:
            # Execute class definition safely
            local_vars = {}
            exec(code, {"np": np}, local_vars)
            cls = local_vars[entry["name"]]
            func = cls(dim=dim).f

            # Evaluate f on 2D grid
            Z = np.zeros_like(X)
            for ix in range(grid_points):
                for iy in range(grid_points):
                    Z[iy, ix] = func([X[iy, ix], Y[iy, ix]])

            # Plot
            cs = ax.contourf(X, Y, Z, levels=50, cmap="viridis")
            #ax.set_title(textwrap.shorten(entry["description"], width=60, placeholder="…"), fontsize=9)
            ax.set_xticks([])
            ax.set_yticks([])
        except Exception as e:
            ax.text(0.5, 0.5, f"Error:\n{e}", ha="center", va="center")
            ax.set_xticks([])
            ax.set_yticks([])

    # Hide unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    #fig.suptitle(f"2D Contour Landscapes - {title}", fontsize=14)
    plt.tight_layout()
    plt.show()
            

## Separable GlobalLocal